In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sentence_transformers import SentenceTransformer

c:\Users\Asush\Notebooks\Service Ticket Call Classification\ve\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
df = pd.read_excel('output.xlsx')
df.head()

,Product,final_problem,Solution
0,Credit card,i am dissatisfied with the current outcome of ...,We apologize for the inconvenience you've expe...
1,"""Credit reporting",incorrect information on your report,We understand your concern about incorrect inf...
2,Bank account or service,on my bank account balance with pnc bank was ...,We regret the inconvenience caused by the over...
3,Debt collection,i received a collection notice and called rig...,We understand your concern about the collectio...
4,Credit card or prepaid card,i had accounts with chase they closed my acco...,We apologize for the inconvenience caused by t...


In [3]:
df_cleaned=df
pd.set_option('display.max_colwidth', None)
df_cleaned=df.iloc[0:30,:]
df_cleaned.shape

(30, 3)

In [4]:
df_cleaned['Product'] = df_cleaned['Product'].replace('"Credit reporting', 'Credit reporting')
df_cleaned['Product'] = df_cleaned['Product'].replace('Credit card', 'Credit card or prepaid card')
df_cleaned['Product'] = df_cleaned['Product'].replace('Prepaid card', 'Credit card or prepaid card')
df_cleaned['Product'] = df_cleaned['Product'].replace('Consumer Loan', 'Bank account or service')
df_cleaned['Product'] = df_cleaned['Product'].replace('Student loan', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Payday loan', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('"Payday loan', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Vehicle loan or lease', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Mortgage', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('Debt collection', 'Loan and debt')
df_cleaned['Product'] = df_cleaned['Product'].replace('"Money transfer', 'Money transfers')
df_cleaned['Product'] = df_cleaned['Product'].replace('Checking or savings account', 'Bank account or service')
df_cleaned['Product'] = df_cleaned['Product'].replace('Virtual currency', 'Other financial service')
df_cleaned['Product'].unique()

C:\Users\Asush\AppData\Local\Temp\ipykernel_17800\2719090875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Product'] = df_cleaned['Product'].replace('"Credit reporting', 'Credit reporting')
C:\Users\Asush\AppData\Local\Temp\ipykernel_17800\2719090875.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Product'] = df_cleaned['Product'].replace('Credit card', 'Credit card or prepaid card')
C:\Users\Asush\AppData\Local\Temp\ipykernel_17800\2719090875.py:3: SettingWithCopyWarning: 

array(['Credit card or prepaid card', 'Credit reporting',
       'Bank account or service', 'Loan and debt'], dtype=object)

Finding type of product for the input value first
{0: 'Bank account or service', 1: 'Credit card or prepaid card', 2: 'Credit reporting', 3: 'Loan and debt', 4: 'Money transfers', 5: 'Other financial service'}



In [5]:
vectorizer_filename = 'tfidf_vectorizer.pkl'
with open(vectorizer_filename, 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

In [6]:
input_data = ["I received a past-due notice for my car loan and immediately contacted the lender, thinking I had simply missed a payment due to our numerous household expenses. During the call, the representative informed me of additional overdue payments dating back several months. I promptly paid the recent notice but explained that I needed to review my records for the older ones. One of these overdue payments was linked to a loan my spouse had taken out, which could not be discussed with me due to privacy regulations. I am now filing a separate complaint to address this issue."]

# Vectorize the input data
X = loaded_vectorizer.transform(input_data) 


In [7]:
import pickle

model_filename = 'lr_tfidf_model.pkl'

with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)

print("Model loaded successfully")

Model loaded successfully


In [8]:
category_mapping = {
    0: 'Bank account or service',
    1: 'Credit card or prepaid card',
    2: 'Credit reporting',
    3: 'Loan and debt',
    4: 'Money transfers',
    5: 'Other financial service'
}
i=loaded_model.predict(X)
category=[category_mapping[label] for label in i][0]
print(category)

Loan and debt


getting all elements of selected category and then using embedding on them

In [14]:

df_filtered = df_cleaned[df_cleaned['Product'] == category]
df_filtered.head()
problems_of_same_category=df_filtered['final_problem']
soultion_for_similar_problems=df_filtered['Solution']

print(problems_of_same_category)
print(soultion_for_similar_problems)

3                                                                                                                                                         i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not be discussed with me a separate complaint is being filed 
5                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [15]:
model = SentenceTransformer("all-mpnet-base-v2")

query_embedding = model.encode(input_data[0])
passage_embeddings = model.encode([
   x for x in problems_of_same_category.tolist()
])

similarity = model.similarity(query_embedding, passage_embeddings)
# Your similarity tensor
similarity_tensor = np.array(similarity)

# Convert the tensor to a numpy array (if not already)
similarity = similarity_tensor.flatten() 
top_indices = np.argsort(similarity)[-3:][::-1]
similar_problem_with_solution={}

for i in top_indices:
    similar_problem_with_solution[problems_of_same_category.tolist()[i]]=soultion_for_similar_problems.tolist()[i]
    



# top_passages = [problems_of_same_category.tolist()[i] for i in top_indices]
# top_scores = [similarity[i] for i in top_indices]

print("Top 3 passages based on similarity scores:")
print(similar_problem_with_solution)
print(similarity)

Top 3 passages based on similarity scores:
{' i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not be discussed with me a separate complaint is being filed ': "We understand your concern about the collection notice you received. It's essential to verify the dates and details of the debts mentioned. For any debts that you are unsure of or that belong to another person, we recommend contacting the collection agency directly. They can provide specific information and guidance on how to proceed with resolving each notice. If you have any further questions or need assistance, please feel free to reach out to us.", 'i am a victim of identity theft company took me to court without notifying me as the sent everything

# Glove

In [13]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Sample corpus
# print(problems_of_same_category.tolist())
documents =input_data
documents.extend(problems_of_same_category.tolist())
print(documents)
documents_df=pd.DataFrame(documents,columns=['documents'])

# removing special characters and stop words from the text
stop_words_l=stopwords.words('english')
documents_df['documents_cleaned']=documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

tfidfvectoriser=TfidfVectorizer()
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(documents_df.documents_cleaned)

['I received a past-due notice for my car loan and immediately contacted the lender, thinking I had simply missed a payment due to our numerous household expenses. During the call, the representative informed me of additional overdue payments dating back several months. I promptly paid the recent notice but explained that I needed to review my records for the older ones. One of these overdue payments was linked to a loan my spouse had taken out, which could not be discussed with me due to privacy regulations. I am now filing a separate complaint to address this issue.', ' i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not be discussed with me a separate complaint is being filed ', ' on or about through ', '

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents = tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_padded_documents = pad_sequences(tokenized_documents, maxlen=64, padding='post')
vocab_size = len(tokenizer.word_index) + 1

# Reading GloVe word embeddings into a dictionary
embeddings_index = {}
with open('D:\\glove.6B\\glove.6B.100d.txt', encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Creating embedding matrix
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Ensure TF-IDF vectorizer has the same number of features as words
# tfidf_vectors = tfidfvectoriser.transform(documents_df.documents_cleaned)
words = tfidfvectoriser.get_feature_names_out()

# Check dimensions for debug
print(f"TF-IDF vector shape: {tfidf_vectors.shape}")
print(f"Words count: {len(words)}")

# Calculating average of word vectors of a document weighted by tf-idf
document_embeddings = np.zeros((documents_df.shape[0], 100))

for i in range(documents_df.shape[0]):
    for j in range(len(words)):
        word = words[j]
        if word in tokenizer.word_index:
            word_index = tokenizer.word_index[word]
            embedding_vector = embedding_matrix[word_index]
            tfidf_value = tfidf_vectors[i, j]
            document_embeddings[i] += embedding_vector * tfidf_value

# Calculating pairwise similarities and differences
pairwise_similarities = cosine_similarity(document_embeddings)
pairwise_differences = euclidean_distances(document_embeddings)

def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print (f'Document: {documents_df.iloc[ix]["documents"]}')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')

most_similar(0,pairwise_similarities,'Cosine Similarity')
most_similar(0,pairwise_differences,'Euclidean Distance')  

TF-IDF vector shape: (13, 187)
Words count: 187
Document: I received a past-due notice for my car loan and immediately contacted the lender, thinking I had simply missed a payment due to our numerous household expenses. During the call, the representative informed me of additional overdue payments dating back several months. I promptly paid the recent notice but explained that I needed to review my records for the older ones. One of these overdue payments was linked to a loan my spouse had taken out, which could not be discussed with me due to privacy regulations. I am now filing a separate complaint to address this issue.


Similar Documents:


Document:  i received a collection notice and called right away i figured i had over looked it as we have many medical bills while on the phone i was told there were additional much older than the notice i called about i paid the one notice and said i needed to look at the date for the other one one of those were for my adult son and could not 